# Pipeline processing using Dask

This notebook demonstrates the continuum imaging and ICAL pipelines.

In [ ]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from processing_components.graphs.execute import arlexecute

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy


import pprint

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from libs.calibration.solvers import solve_gaintable
from libs.calibration.operations import apply_gaintable
from libs.calibration.calibration_control import create_calibration_controls
from libs.data.data_models import Image
from libs.data.polarisation import PolarisationFrame
from libs.data.parameters import get_parameter
from libs.visibility.base import create_blockvisibility
from libs.skycomponent.operations import create_skycomponent
from libs.image.deconvolution import deconvolve_cube
from libs.image.operations import show_image, export_image_to_fits, qa_image, copy_image, create_empty_image_like
from libs.visibility.iterators import vis_timeslice_iter
from libs.util.testing_support import create_named_configuration, create_low_test_beam
from libs.imaging import predict_2d, create_image_from_visibility, advise_wide_field

from processing_components.graphs.dask_init import get_dask_Client
from processing_components.graphs.imaging import create_invert_graph, create_predict_graph, create_deconvolve_graph, \
    create_residual_graph, compute_list
from processing_components.graphs.generic_graphs import create_generic_image_graph
from processing_components.graphs.graph_support import create_simulate_vis_graph, \
    create_low_test_image_from_gleam, create_corrupt_vis_graph
from processing_components.graphs.pipeline_graphs import create_continuum_imaging_pipeline_graph, \
    create_ical_pipeline_graph    
    
import pprint
pp = pprint.PrettyPrinter()

import logging
def init_logging():
    log = logging.getLogger()
    logging.basicConfig(filename='%s/imaging-pipeline.log' % results_dir,
                            filemode='a',
                            format='%(thread)s %(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.INFO)
log = logging.getLogger()
logging.info("Starting imaging-pipeline")

We will use dask

In [ ]:
arlexecute.set_client(get_dask_Client())
arlexecute.client.run(init_logging)

In [ ]:
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

We create a graph to make the visibility. The parameter rmax determines the distance of the furthest antenna/stations used. All over parameters are determined from this number.

In [ ]:
nfreqwin=7
ntimes=11
rmax=300.0
frequency=numpy.linspace(0.9e8,1.1e8,nfreqwin)
channel_bandwidth=numpy.array(nfreqwin*[frequency[1]-frequency[0]])
times = numpy.linspace(-numpy.pi/3.0, numpy.pi/3.0, ntimes)
phasecentre=SkyCoord(ra=+30.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox='J2000')

vis_graph_list=create_simulate_vis_graph('LOWBD2',
                                         frequency=frequency, 
                                         channel_bandwidth=channel_bandwidth,
                                         times=times,
                                         phasecentre=phasecentre,
                                         order='frequency',
                                        rmax=rmax)
print('%d elements in vis_graph_list' % len(vis_graph_list))
log.info('About to make visibility')
vis_graph_list = arlexecute.compute(vis_graph_list)

In [ ]:
wprojection_planes=1
advice_low=advise_wide_field(vis_graph_list[0], guard_band_image=8.0, delA=0.02,
                             wprojection_planes=wprojection_planes)

advice_high=advise_wide_field(vis_graph_list[-1], guard_band_image=8.0, delA=0.02,
                              wprojection_planes=wprojection_planes)

vis_slices = advice_low['vis_slices']
npixel=advice_high['npixels2']
cellsize=min(advice_low['cellsize'], advice_high['cellsize'])

Now make a graph to fill with a model drawn from GLEAM 

In [ ]:
gleam_model_graph = [arlexecute.execute(create_low_test_image_from_gleam)(npixel=npixel,
                                                               frequency=[frequency[f]],
                                                               channel_bandwidth=[channel_bandwidth[f]],
                                                               cellsize=cellsize,
                                                               phasecentre=phasecentre,
                                                               polarisation_frame=PolarisationFrame("stokesI"),
                                                               flux_limit=1.0,
                                                               applybeam=True)
                     for f, freq in enumerate(frequency)]
log.info('About to make GLEAM model')
gleam_model_graph = arlexecute.compute(gleam_model_graph)
future_gleam_model_graph = arlexecute.client.scatter(gleam_model_graph)

In [ ]:
log.info('About to run predict to get predicted visibility')
future_vis_graph = arlexecute.client.scatter(vis_graph_list)
predicted_vis_graph_list = create_predict_graph(future_vis_graph, future_gleam_model_graph,  
                                                context='wstack', vis_slices=vis_slices)
predicted_vis_graph_list = arlexecute.compute(predicted_vis_graph_list)
corrupted_vis_graph_list = create_corrupt_vis_graph(predicted_vis_graph_list, phase_error=1.0)
log.info('About to run corrupt to get corrupted visibility')
corrupted_vis_graph_list =  arlexecute.compute(corrupted_vis_graph_list)
future_predicted_vis_graph_list=arlexecute.client.scatter(predicted_vis_graph_list)

Get the LSM. This is currently blank.

In [ ]:
model_graph = [arlexecute.execute(create_image_from_visibility)(vis_graph_list[f],
                                                     npixel=npixel,
                                                     frequency=[frequency[f]],
                                                     channel_bandwidth=[channel_bandwidth[f]],
                                                     cellsize=cellsize,
                                                     phasecentre=phasecentre,
                                                     polarisation_frame=PolarisationFrame("stokesI"))
               for f, freq in enumerate(frequency)]

In [ ]:
future_predicted_vis_graph_list=arlexecute.client.scatter(predicted_vis_graph_list)
dirty_graph = create_invert_graph(future_predicted_vis_graph_list, model_graph, 
                                  context='wstack',
                                  vis_slices=vis_slices, dopsf=False)
psf_graph = create_invert_graph(future_predicted_vis_graph_list, model_graph, 
                                context='wstack',
                                vis_slices=vis_slices, dopsf=True)

Create and execute graphs to make the dirty image and PSF

In [ ]:
log.info('About to run invert to get dirty image')

dirty_graph =  arlexecute.compute(dirty_graph)
dirty = dirty_graph[0][0]
print(dirty)
show_image(dirty, cm='Greys', vmax=1.0, vmin=-0.1)
plt.show()

log.info('About to run invert to get PSF')


psf_graph =  arlexecute.compute(psf_graph)
psf = psf_graph[0][0]
show_image(psf, cm='Greys', vmax=0.1, vmin=-0.01)
plt.show()

Now deconvolve using msclean

In [ ]:
log.info('About to run deconvolve')

deconvolve_graph, _ = \
    create_deconvolve_graph(dirty_graph, psf_graph, model_graph=model_graph, 
                            deconvolve_facets=8, deconvolve_overlap=16, deconvolve_taper='tukey',
                            scales=[0, 3, 10],
                            algorithm='msclean', niter=1000, 
                            fractional_threshold=0.1,
                            threshold=0.1, gain=0.1, psf_support=64)
    
deconvolve_graph.visualize('%s/imaging-pipelines-deconvolve-facet-graph.svg' %(results_dir))

deconvolved = arlexecute.compute(deconvolve_graph)
show_image(deconvolved[0], cm='Greys', vmax=0.1, vmin=-0.01)
plt.show()

In [ ]:
continuum_imaging_graph = \
    create_continuum_imaging_pipeline_graph(predicted_vis_graph_list, 
                                            model_graph=model_graph, 
                                            context='wstack', vis_slices=vis_slices, 
                                            scales=[0, 3, 10], algorithm='mmclean', 
                                            nmoment=3, niter=1000, 
                                            fractional_threshold=0.1,
                                            threshold=0.1, nmajor=5, gain=0.25,
                                            deconvolve_facets = 8, deconvolve_overlap=16, 
                                            deconvolve_taper='tukey', psf_support=64)


In [ ]:
log.info('About to run continuum imaging')

result=arlexecute.compute(continuum_imaging_graph)
deconvolved = result[0][0]
residual = result[1][0]
restored = result[2][0]

f=show_image(deconvolved, title='Clean image - no selfcal', cm='Greys', 
             vmax=0.1, vmin=-0.01)
print(qa_image(deconvolved, context='Clean image - no selfcal'))

plt.show()

f=show_image(restored, title='Restored clean image - no selfcal', 
             cm='Greys', vmax=1.0, vmin=-0.1)
print(qa_image(restored, context='Restored clean image - no selfcal'))
plt.show()
export_image_to_fits(restored, '%s/imaging-dask_continuum_imaging_restored.fits' 
                     %(results_dir))

f=show_image(residual[0], title='Residual clean image - no selfcal', cm='Greys', 
             vmax=0.1, vmin=-0.01)
print(qa_image(residual[0], context='Residual clean image - no selfcal'))
plt.show()
export_image_to_fits(residual[0], '%s/imaging-dask_continuum_imaging_residual.fits' 
                     %(results_dir))

In [ ]:
for chan in range(nfreqwin):
    residual = result[1][chan]
    show_image(residual[0], title='Channel %d' % chan, cm='Greys', 
               vmax=0.1, vmin=-0.01)
    plt.show()

In [ ]:
controls = create_calibration_controls()
        
controls['T']['first_selfcal'] = 1
controls['G']['first_selfcal'] = 3
controls['B']['first_selfcal'] = 4

controls['T']['timescale'] = 'auto'
controls['G']['timescale'] = 'auto'
controls['B']['timescale'] = 1e5

pp.pprint(controls)

In [ ]:
future_corrupted_vis_graph_list = arlexecute.client.scatter(corrupted_vis_graph_list)
ical_graph = create_ical_pipeline_graph(future_corrupted_vis_graph_list, 
                                        model_graph=model_graph,  
                                        context='wstack', 
                                        calibration_context = 'TG', 
                                        controls=controls,
                                        scales=[0, 3, 10], algorithm='mmclean', 
                                        nmoment=3, niter=1000, 
                                        fractional_threshold=0.1,
                                        threshold=0.1, nmajor=5, gain=0.25,
                                        deconvolve_facets = 8, 
                                        deconvolve_overlap=16,
                                        deconvolve_taper='tukey',
                                        vis_slices=ntimes,
                                        timeslice='auto',
                                        global_solution=False, 
                                        psf_support=64,
                                        do_selfcal=True)

In [ ]:
log.info('About to run ical')
result=arlexecute.compute(ical_graph)
deconvolved = result[0][0]
residual = result[1][0]
restored = result[2][0]

f=show_image(deconvolved, title='Clean image', cm='Greys', vmax=1.0, vmin=-0.1)
print(qa_image(deconvolved, context='Clean image'))
plt.show()

f=show_image(restored, title='Restored clean image', cm='Greys', vmax=1.0, 
             vmin=-0.1)
print(qa_image(restored, context='Restored clean image'))
plt.show()
export_image_to_fits(restored, '%s/imaging-dask_ical_restored.fits' 
                     %(results_dir))

f=show_image(residual[0], title='Residual clean image', cm='Greys', 
             vmax=0.1, vmin=-0.01)
print(qa_image(residual[0], context='Residual clean image'))
plt.show()
export_image_to_fits(residual[0], '%s/imaging-dask_ical_residual.fits' 
                     %(results_dir))